## Importando Bibliotecas

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np


In [3]:
df = pd.read_csv("/Users/lucas/TCC/deputados_federais_com_financiamento.csv")

## Limpeza

In [4]:
df['Decision'] = 'NaN'

In [5]:
for index, row in df.iterrows():
    if row['DS_SIT_TOT_TURNO'] == "SUPLENTE" or row['DS_SIT_TOT_TURNO'] == "N�O ELEITO" or row['DS_SIT_TOT_TURNO'] == "#NULO#" :
        df.loc[index,'Decision'] = 0
    else: df.loc[index,'Decision'] = 1

In [6]:
## fatorando

df['SG_PARTIDO_F'] = pd.factorize(df['SG_PARTIDO'])[0]
df['ST_REELEICAO_F'] = pd.factorize(df['ST_REELEICAO'])[0]
df['ST_DECLARAR_BENS_F'] = pd.factorize(df['ST_DECLARAR_BENS'])[0]


In [7]:
## Utilizando todo o sample

sample = df[["CD_SITUACAO_CANDIDATURA","SQ_COLIGACAO","NR_IDADE_DATA_POSSE","CD_GENERO","CD_ESTADO_CIVIL",
             "CD_COR_RACA","CD_OCUPACAO","VR_DESPESA_MAX_CAMPANHA","FIN_COMERCIALIZACAO_DE_BENS_EVENTOS",
             "FIN_DOACAO_PELA_INTERNET","FIN_RECURSOS_FINANCIAMENTO_COLETIVO","FIN_ORIGEM_NAO_IDENTIFICADA",
             "FIN_RECURSOS_OUTROS_CANDIDATOS","FIN_PARTIDO_POLITICO","FIN_RECURSOS_PESSOAS_FISICAS",
             "RECURSOS_PROPRIOS","FIN_APLICACOES_FINANCEIRAS","SG_PARTIDO_F","ST_REELEICAO_F","ST_DECLARAR_BENS_F","Decision"]]

In [8]:
sample.dropna(inplace=True)

c:\users\lucas\appdata\local\programs\python\python38\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [9]:
sample.Decision=sample.Decision.astype('int')

c:\users\lucas\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [10]:
# Dividindo os dados em treino e teste:
X_train, X_test, y_train, y_test = train_test_split(sample.drop('Decision',axis=1),sample['Decision'],test_size=0.3)

In [11]:
X_train.shape,X_test.shape

((6011, 20), (2577, 20))

In [12]:
y_train.shape,y_test.shape

((6011,), (2577,))

In [13]:
y_train=y_train.astype('int')
y_test=y_test.astype('int')

## Usando SKlearn

https://minerandodados.com.br/arvores-de-decisao-conceitos-e-aplicacoes/

In [14]:
# Instânciando o objeto classificador:
clf = DecisionTreeClassifier()

In [15]:
# Treinando o modelo de arvore de decisão:
clf = clf.fit(X_train,y_train)

In [16]:
# Verificando as features mais importantes para o modelo treinado:
clf.feature_importances_

array([0.00525204, 0.03973571, 0.07155465, 0.01003517, 0.01417273,
       0.00678463, 0.05854393, 0.        , 0.00246027, 0.        ,
       0.03553663, 0.00204283, 0.05110377, 0.3846437 , 0.11475087,
       0.09558367, 0.01052736, 0.06986542, 0.02321157, 0.00419509])

In [27]:
for feature,importancia in zip(sample.columns,clf.feature_importances_):
    print("{}:{}".format(feature, importancia))

CD_SITUACAO_CANDIDATURA:0.0025817380793465125
SQ_COLIGACAO:0.0637118135000326
NR_IDADE_DATA_POSSE:0.07203609368198578
CD_GENERO:0.002195365059251757
CD_ESTADO_CIVIL:0.012876573800596794
CD_COR_RACA:0.01588381006472606
CD_OCUPACAO:0.05052163302484052
VR_DESPESA_MAX_CAMPANHA:0.0
FIN_COMERCIALIZACAO_DE_BENS_EVENTOS:0.002295395791285064
FIN_DOACAO_PELA_INTERNET:0.00035656533185121885
FIN_RECURSOS_FINANCIAMENTO_COLETIVO:0.023136040563460025
FIN_ORIGEM_NAO_IDENTIFICADA:0.007174055901540848
FIN_RECURSOS_OUTROS_CANDIDATOS:0.02949044511414485
FIN_PARTIDO_POLITICO:0.38321603405371046
FIN_RECURSOS_PESSOAS_FISICAS:0.14223902888863066
RECURSOS_PROPRIOS:0.09135343358184332
FIN_APLICACOES_FINANCEIRAS:0.00894087240009892
SG_PARTIDO_F:0.06239046846639272
ST_REELEICAO_F:0.029206914167216692
ST_DECLARAR_BENS_F:0.00039371852904504143


In [28]:
resultado = clf.predict(X_test)
resultado

array([0, 1, 0, ..., 0, 0, 0])

## metricas

In [29]:
from sklearn import metrics
print(metrics.classification_report(y_test,resultado))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2433
           1       0.48      0.49      0.48       144

    accuracy                           0.94      2577
   macro avg       0.72      0.73      0.73      2577
weighted avg       0.94      0.94      0.94      2577



## arvore de forma grafica

In [32]:
# Renderizando a árvore de forma gráfica:
!pip install ipywidgets
!pip3 install graphviz
!pip3 install pydot
import pydot
import graphviz

dot_data = export_graphviz( 
         clf, 
         out_file=None,
         feature_names=sample.drop('DD',axis=1).columns,
         class_names=['0','1'],  
         filled=True, rounded=True,
         proportion=True,
         node_ids=True,
         rotate=False,
         label='all',
         special_characters=True
        )  
graph = graphviz.Source(dot_data)  
graph

KeyError: "['Outcome'] not found in axis"

In [17]:
from ipywidgets import interactive
from IPython.display import SVG,display
from graphviz import Source

In [20]:
import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'

In [18]:
# feature matrix
X,y = sample.drop('Decision',axis=1),sample['Decision']

# feature labels
features_label = sample.drop('Decision',axis=1).columns

# class label
class_label = ['0','1']

In [21]:
def plot_tree(crit, split, depth, min_samples_split, min_samples_leaf=0.2):
    estimator = DecisionTreeClassifier(
           random_state = 0 
          ,criterion = crit
          ,splitter = split
          ,max_depth = depth
          ,min_samples_split=min_samples_split
          ,min_samples_leaf=min_samples_leaf
    )
    estimator.fit(X, y)
    graph = Source(export_graphviz(estimator
      , out_file=None
      , feature_names=features_label
      , class_names=class_label
      , impurity=True
      , filled = True))
    display(SVG(graph.pipe(format='svg')))
    return estimator

inter=interactive(plot_tree 
   , crit = ["gini", "entropy"]
   , split = ["best", "random"]
   , depth=[1,2,3,4,5,10,20,30]
   , min_samples_split=(1,5)
   , min_samples_leaf=(1,5))

display(inter)

interactive(children=(Dropdown(description='crit', options=('gini', 'entropy'), value='gini'), Dropdown(descri…